Playing around with symbolic maths in Julia I realized how good SymPy is compared to other symbolic computation libraries. I came across [this](https://mobile.twitter.com/ChadScherrer/status/1215479229866921984) tweet which shows the power of SymPy with julia MetaProgramming. 

In [2]:
import PyCall, SymPy
using MLStyle
stats = PyCall.pyimport_conda("sympy.stats", "sympy")
SymPy.import_from(stats)


In [3]:
sym(x) = SymPy.symbols(x)

macro ℓ(expr)
    args = @match expr begin
        Expr(head, args...) => return args
    end

    d = args[1]
    ps = args[2:end]

    quote 
        L = SymPy.density(stats.$d(:foo, sym.($ps)...)).pdf(sym(:x))
        SymPy.sympy.expand_log(log(L), force=true)
    end
end


@ℓ (macro with 1 method)

This macro takes any expression like `Poisson(λ)` or `Normal(μ, σ)` and gives a log-likelihood expression for it. 

In [4]:
@ℓ Poisson(λ)

x⋅log(λ) - λ - log(x!)

In [5]:
@ℓ Normal(μ, σ)

                                   2
          log(π)   log(2)   (x - μ) 
-log(σ) - ────── - ────── - ────────
            2        2           2  
                              2⋅σ   

In [27]:
SymPy.N((@ℓ Poisson(λ))(10)(1))

-7.697414907005954315982008545315635792398898511371227023966672099032427390322668